# Mounting Google Drive

In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [1]:
!pip install ffn

In [140]:
import math
import json
import os
import ffn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import LSTM, GRU
from keras.models import Sequential, load_model
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.backend as K
import scipy
import pandas as pd
# from pandas_datareader import data as pdr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

print("-----------------------------")
print(f'Tensorflow version: {tf.__version__}')
# print(f'Matplotlib version: {matplotlib.__version__}')
print("All neccessary imports done!")

-----------------------------
Tensorflow version: 2.15.0
All neccessary imports done!


# Variables

In [11]:
# Define the list of assets
assets = ["AAPL", "ABBV", "ABT", "ACN", "ADBE", "ADI", "ADP", "AMAT", "AMD", "AMGN", "AMZN", "AVGO", "AXP", "BA", "BAC", "BKNG", "BLK", "BRK-B", "BSX", "C", "CAT", "CB", "CI", "CMCSA", "COP", "COST", "CRM", "CSCO", "CVX", "DE", "DHR", "DIS", "ELV", "ETN", "FI", "GE", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "INTU", "ISRG", "JNJ", "JPM", "KLAC", "KO", "LIN", "LLY", "LMT", "LOW", "LRCX", "MA", "MCD", "MDLZ", "MDT", "META", "MMC", "MRK", "MS", "MSFT", "MU", "NEE", "NFLX", "NKE", "NOW", "NVDA", "ORCL", "PANW", "PEP", "PFE", "PG", "PGR", "PLD", "PM", "QCOM", "REGN", "RTX", "SBUX", "SCHW", "SNPS", "SPGI", "SYK", "T", "TJX", "TMO", "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "V", "VRTX", "VZ", "WFC", "WMT", "XOM"]

functions = ['BALANCE_SHEET', 'CASH_FLOW', 'EARNINGS', 'INCOME_STATEMENT']

In [5]:
# sorting tickers
assets.sort()

time_steps = 100 # the lookback period
features = 1 # We use only the close price
min_prev_days = 252
port_len = len(assets) # length of portfolio
max_port_len = 100 # maximum length of portfolio that is allowed

In [19]:
for asset in assets:
  filename = '/content/drive/MyDrive/financial_data/financial_data_' + str(asset) + '.json'

  with open(filename, 'r') as f:
    data = json.load(f)

  for function in functions:
    if function == 'EARNINGS':
      func_list = data[function]['quarterlyEarnings']
    else:
      func_list = data[function]['quarterlyReports']
    df = pd.DataFrame(columns = func_list[0].keys())
    for i, entry_dict in enumerate(func_list):
      df.loc[i] = entry_dict
    df = df.set_index('fiscalDateEnding')
    df.index = pd.to_datetime(df.index)
    df.to_csv(f'/content/drive/MyDrive/sp500/{asset}_{function}.csv')

In [34]:
df = pd.read_csv('/content/drive/MyDrive/Internship Project/Datasets/sp500_close_data.csv')
df = df.set_index('Date').dropna()
df.index = pd.to_datetime(df.index)
df

,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADP,AMAT,AMD,AMGN,...,TXN,UNH,UNP,UPS,V,VRTX,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2014-05-30,22.607143,54.330002,40.009998,81.449997,64.540001,52.380001,69.956100,20.190001,4.000000,115.989998,...,46.980000,79.629997,99.635002,103.879997,53.707500,72.260002,49.959999,50.779999,25.590000,100.529999
2014-06-02,22.451786,54.150002,39.799999,81.269997,64.639999,52.369999,69.692711,20.520000,3.970000,116.449997,...,46.990002,79.470001,99.584999,104.080002,53.380001,72.629997,50.049999,51.090000,25.586666,99.940002
2014-06-03,22.769285,54.369999,39.820000,81.870003,64.089996,52.470001,69.104477,21.420000,3.940000,117.120003,...,46.990002,79.879997,98.695000,103.430000,52.830002,72.959999,49.290001,51.090000,25.570000,100.389999
2014-06-04,23.029285,54.580002,39.750000,83.449997,64.169998,52.490002,69.209831,21.559999,4.040000,119.150002,...,46.849998,80.510002,98.915001,102.970001,52.857498,72.639999,49.150002,51.040001,25.709999,100.040001
2014-06-05,23.119642,55.299999,40.150002,83.580002,65.470001,52.570000,69.095695,21.780001,4.080000,117.739998,...,47.400002,79.800003,100.169998,103.620003,53.055000,73.059998,49.279999,51.630001,25.773333,100.550003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-23,186.880005,158.259995,104.230003,305.929993,483.309998,234.559998,251.490005,217.949997,160.429993,306.500000,...,197.289993,516.830017,232.710007,139.100006,274.230011,450.799988,39.430000,59.680000,64.839996,113.510002
2024-05-24,189.979996,157.059998,103.949997,300.989990,475.429993,232.509995,248.899994,220.889999,166.360001,305.839996,...,199.179993,508.170013,232.050003,138.660004,274.489990,456.950012,39.740002,60.209999,65.379997,113.419998
2024-05-28,189.990005,155.339996,101.949997,297.730011,478.429993,233.440002,243.300003,221.320007,171.610001,300.190002,...,199.600006,503.679993,229.009995,137.580002,270.980011,446.880005,39.360001,59.480000,65.040001,114.860001


In [26]:
extra_info_tickers = {}
for ticker in assets:
  extra_info_tickers[ticker] = {}

  extra_info_tickers[ticker]['earnings'] = pd.read_csv(f'/content/drive/MyDrive/sp500/{ticker}_EARNINGS.csv')
  extra_info_tickers[ticker]['earnings'] = extra_info_tickers[ticker]['earnings'].set_index('fiscalDateEnding')
  extra_info_tickers[ticker]['earnings'].index = pd.to_datetime(extra_info_tickers[ticker]['earnings'].index)

  extra_info_tickers[ticker]['financials'] = pd.read_csv(f'/content/drive/MyDrive/sp500/{ticker}_INCOME_STATEMENT.csv')
  extra_info_tickers[ticker]['financials'] = extra_info_tickers[ticker]['financials'].set_index('fiscalDateEnding')
  extra_info_tickers[ticker]['financials'].index = pd.to_datetime(extra_info_tickers[ticker]['financials'].index)

  extra_info_tickers[ticker]['balance_sheet'] = pd.read_csv(f'/content/drive/MyDrive/sp500/{ticker}_BALANCE_SHEET.csv')
  extra_info_tickers[ticker]['balance_sheet'] = extra_info_tickers[ticker]['balance_sheet'].set_index('fiscalDateEnding')
  extra_info_tickers[ticker]['balance_sheet'].index = pd.to_datetime(extra_info_tickers[ticker]['balance_sheet'].index)

  extra_info_tickers[ticker]['cash_flow'] = pd.read_csv(f'/content/drive/MyDrive/sp500/{ticker}_CASH_FLOW.csv')
  extra_info_tickers[ticker]['cash_flow'] = extra_info_tickers[ticker]['cash_flow'].set_index('fiscalDateEnding')
  extra_info_tickers[ticker]['cash_flow'].index = pd.to_datetime(extra_info_tickers[ticker]['cash_flow'].index)

In [29]:
extra_info_tickers['AAPL']['earnings'].T

fiscalDateEnding,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,...,1998-06-30,1998-03-31,1997-12-31,1997-09-30,1997-06-30,1997-03-31,1996-12-31,1996-09-30,1996-06-30,1996-03-31
reportedDate,2024-05-02,2024-02-01,2023-11-02,2023-08-03,2023-05-04,2023-02-02,2022-10-27,2022-07-28,2022-04-28,2022-01-27,...,1998-07-15,1998-04-15,1998-01-14,1997-10-15,1997-07-16,1997-04-16,1997-01-15,1996-10-16,1996-07-17,1996-04-17
reportedEPS,1.53,2.18,1.46,1.26,1.52,1.88,1.29,1.2,1.52,2.1,...,0.01,0.01,0.01,-0.0114,-0.02,-0.05,-0.03,0.0018,-0.02,-0.07
estimatedEPS,1.5,2.1,1.39,1.19,1.43,1.94,1.27,1.16,1.43,1.89,...,0.01,0.01,0.01,NaN,-0.02,-0.04,-0.02,-0.01,-0.04,-0.05
surprise,0.03,0.08,0.07,0.07,0.09,-0.06,0.02,0.04,0.09,0.21,...,0.0,0.0,0.0,NaN,0.0,-0.01,-0.01,0.0118,0.02,-0.02
surprisePercentage,2.0,3.8095,5.036,5.8824,6.2937,-3.0928,1.5748,3.4483,6.2937,11.1111,...,0.0,0.0,0.0,NaN,0.0,-25.0,-50.0,118.0,50.0,-40.0
reportTime,post-market,post-market,post-market,post-market,post-market,post-market,post-market,post-market,post-market,post-market,...,pre-market,pre-market,pre-market,pre-market,pre-market,pre-market,pre-market,pre-market,pre-market,pre-market


In [30]:
extra_info_tickers['AAPL']['balance_sheet'].T

fiscalDateEnding,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,...,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
totalAssets,337411000000,353514000000,352583000000,335038000000,332160000000,346747000000,352755000000,336309000000,350662000000,381191000000,...,116371000000,106758000000,94904000000,86742000000,75183000000,64725000000,57057000000,53926000000,47501000000,48140000000
totalCurrentAssets,128416000000,143692000000,143566000000,122659000000,112913000000,128777000000,135405000000,112292000000,118180000000,153154000000,...,44988000000,46898000000,46997000000,43927000000,41678000000,36033000000,32336000000,33332000000,31555000000,35170000000
cashAndCashEquivalentsAtCarryingValue,32695000000,40760000000,29965000000,28408000000,24687000000,20535000000,23646000000,27502000000,28098000000,37119000000,...,9815000000,12091000000,15978000000,10734000000,11261000000,9705000000,10018000000,7609000000,5263000000,5605000000
cashAndShortTermInvestments,67150000000,73100000000,61555000000,62482000000,24687000000,51355000000,48304000000,48231000000,51511000000,63913000000,...,25952000000,28395000000,29234000000,26977000000,25620000000,24288000000,23155000000,24796000000,23464000000,5605000000
inventory,6232000000,6511000000,6331000000,7351000000,7482000000,6820000000,4946000000,5433000000,5460000000,5876000000,...,776000000,889000000,930000000,885000000,1051000000,942000000,638000000,576000000,455000000,380000000
currentNetReceivables,41150000000.0,50102000000.0,60985000000.0,39186000000.0,NaN,54180000000.0,60932000000.0,42242000000.0,45400000000.0,65253000000.0,...,5369000000.0,6102000000.0,5798000000.0,6027000000.0,5510000000.0,3447000000.0,2886000000.0,3090000000.0,3361000000.0,NaN
totalNonCurrentAssets,208995000000,209822000000,209017000000,212379000000,219247000000,217970000000,217350000000,224017000000,232482000000,228037000000,...,123607000000,105103000000,82751000000,73287000000,57101000000,48281000000,39766000000,32503000000,23520000000,12970000000
propertyPlantEquipment,43546000000,43666000000,43715000000,43550000000,43398000000,42951000000,42117000000,40335000000,39304000000,39245000000,...,7777000000,2531000000,2352000000,2210000000,2174000000,2028000000,3504000000,3115000000,1780000000,2653000000
accumulatedDepreciationAmortizationPPE,71697000000.0,72510000000.0,70884000000.0,70787000000.0,69668000000.0,68044000000.0,72340000000.0,71516000000.0,70020000000.0,68454000000.0,...,3991000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
extra_info_tickers['AAPL']['cash_flow'].T

fiscalDateEnding,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,...,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
operatingCashflow,22690000000,39895000000,21598000000,26380000000,28560000000,28560000000,34005000000,24127000000,22892000000,28166000000,...,17554000000,10429000000,11108000000,6219000000,9773000000,5683000000,4801000000,2330000000,5781000000,3110000000
paymentsForOperatingActivities,NaN,NaN,3113000000.0,NaN,NaN,NaN,703000000.0,2755000000.0,504000000.0,875000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
proceedsFromOperatingActivities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
changeInOperatingLiabilities,-11740000000.0,-8407000000.0,14946000000.0,5203000000.0,-16821000000.0,NaN,-2186000000.0,21420000000.0,-6627000000.0,-23746000000.0,...,8082000000.0,2300000000.0,2366000000.0,365000000.0,3633000000.0,4923000000.0,3107000000.0,-1606000000.0,1878000000.0,NaN
changeInOperatingAssets,-5976000000.0,-9530000000.0,21006000000.0,4454000000.0,-17052000000.0,NaN,-689000000.0,21468000000.0,-3402000000.0,-21694000000.0,...,6189000000.0,-29000000.0,0.0,1594000000.0,1342000000.0,3683000000.0,2341000000.0,-251000000.0,-197000000.0,NaN
depreciationDepletionAndAmortization,2836000000,2848000000,2653000000,3052000000,2898000000,2898000000,2916000000,2865000000,2805000000,2737000000,...,721000000,543000000,481000000,434000000,356000000,329000000,273000000,216000000,209000000,197000000
capitalExpenditures,1996000000,2392000000,2163000000,2093000000,2916000000,2916000000,3787000000,3289000000,2102000000,2514000000,...,1321000000,1645000000,777000000,624000000,1214000000,760000000,595000000,274000000,376000000,459000000
changeInReceivables,-8767000000.0,-11124000000.0,21137000000.0,3661000000.0,-17786000000.0,17786000000.0,-6595000000.0,18693000000.0,-3165000000.0,-19931000000.0,...,4767000000.0,244000000.0,376000000.0,221000000.0,950000000.0,3525000000.0,1817000000.0,-211000000.0,-271000000.0,NaN
changeInInventory,-190000000,137000000,-952000000,22000000,741000000,-741000000,1807000000,-435000000,16000000,-384000000,...,460000000,-113000000,-41000000,45000000,-166000000,109000000,304000000,62000000,121000000,75000000


In [32]:
extra_info_tickers['AAPL']['financials'].T

fiscalDateEnding,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,...,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-06-30
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
grossProfit,42271000000,54855000000,40427000000,36413000000,41976000000,50332000000,38095000000,35885000000,42559000000,54243000000,...,7512000000,6136000000,5625000000,6411000000,5105000000,3983000000,3627000000,4507000000,4791000000,2600000000
totalRevenue,90753000000,119575000000,88496000000,80799000000,94836000000,116151000000,89319000000,82240000000,96587000000,123251000000,...,20343000000,15700000000,13499000000,15683000000,12207000000,8337000000,8163000000,10167000000,7895000000,7464000000
costOfRevenue,54950000000,71506000000,55222000000,51357000000,59061000000,73429000000,58491000000,53086000000,60912000000,76151000000,...,15217000000,11002000000,9094000000,10560000000,7102000000,5314000000,5192000000,6635000000,5156000000,4864000000
costofGoodsAndServicesSold,48482000000,64720000000,49071000000,45384000000,52860000000,66822000000,52051000000,47074000000,54719000000,69702000000,...,12831000000,9564000000,7874000000,9272000000,7102000000,5751000000,5457000000,7373000000,8116000000,4864000000
operatingIncome,27900000000,40373000000,26969000000,22998000000,28318000000,36016000000,24894000000,23076000000,29979000000,41488000000,...,5447000000,4234000000,3979000000,4725000000,3684000000,2632000000,2323000000,3101000000,3494000000,1392000000
sellingGeneralAndAdministrative,6468000000,6786000000,6151000000,5973000000,6201000000,6607000000,6440000000,6012000000,6193000000,6449000000,...,1571000000,1438000000,1220000000,1288000000,1063000000,1010000000,985000000,1091000000,999000000,916000000
researchAndDevelopment,7903000000,7696000000,7307000000,7442000000,7457000000,7709000000,6761000000,6797000000,6387000000,6306000000,...,494000000,464000000,426000000,398000000,358000000,341000000,319000000,315000000,298000000,292000000
operatingExpenses,14371000000,14482000000,13458000000,13415000000,13658000000,14316000000,13201000000,12809000000,12580000000,12755000000,...,2065000000,1902000000,1646000000,1686000000,1421000000,1351000000,1304000000,1406000000,1297000000,1208000000
investmentIncomeNet,NaN,NaN,984000000.0,980000000.0,NaN,868000000.0,753000000.0,722000000.0,700000000.0,650000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Using FFN library for performance evaluation of different assets

## Total Returns

In [141]:
for i in range(port_len):
  print(f'Total return of {assets[i]} - {round(ffn.core.calc_total_return(df[assets[i]]) * 100, 2)}%')

Total return of AAPL - 747.92%
Total return of ABBV - 186.77%
Total return of ABT - 155.66%
Total return of ACN - 250.41%
Total return of ADBE - 600.58%
Total return of ADI - 337.73%
Total return of ADP - 244.71%
Total return of AMAT - 981.97%
Total return of AMD - 4081.63%
Total return of AMGN - 159.66%
Total return of AMZN - 1051.02%
Total return of AVGO - 1830.37%
Total return of AXP - 158.07%
Total return of BA - 28.07%
Total return of BAC - 151.98%
Total return of BKNG - 193.36%
Total return of BLK - 149.51%
Total return of BRK-B - 217.36%
Total return of BSX - 486.75%
Total return of C - 30.69%
Total return of CAT - 233.6%
Total return of CB - 156.27%
Total return of CI - 270.18%
Total return of CMCSA - 48.16%
Total return of COP - 42.23%
Total return of COST - 604.63%
Total return of CRM - 311.84%
Total return of CSCO - 88.17%
Total return of CVX - 28.53%
Total return of DE - 302.28%
Total return of DHR - 380.0%
Total return of DIS - 21.16%
Total return of ELV - 369.42%
Total re

## CAGR (Compound Annual Growth Rate)

In [142]:
for i in range(port_len):
  print(f'CAGR of {assets[i]} - {round(ffn.core.calc_cagr(df[assets[i]]) * 100, 2)}%')

CAGR of AAPL - 23.83%
CAGR of ABBV - 11.11%
CAGR of ABT - 9.84%
CAGR of ACN - 13.36%
CAGR of ADBE - 21.49%
CAGR of ADI - 15.91%
CAGR of ADP - 13.17%
CAGR of AMAT - 26.88%
CAGR of AMD - 45.25%
CAGR of AMGN - 10.01%
CAGR of AMZN - 27.67%
CAGR of AVGO - 34.45%
CAGR of AXP - 9.94%
CAGR of BA - 2.5%
CAGR of BAC - 9.68%
CAGR of BKNG - 11.36%
CAGR of BLK - 9.57%
CAGR of BRK-B - 12.24%
CAGR of BSX - 19.35%
CAGR of C - 2.71%
CAGR of CAT - 12.8%
CAGR of CB - 9.87%
CAGR of CI - 13.98%
CAGR of CMCSA - 4.01%
CAGR of COP - 3.59%
CAGR of COST - 21.56%
CAGR of CRM - 15.2%
CAGR of CSCO - 6.53%
CAGR of CVX - 2.54%
CAGR of DE - 14.93%
CAGR of DHR - 16.98%
CAGR of DIS - 1.94%
CAGR of ELV - 16.72%
CAGR of ETN - 16.32%
CAGR of FI - 17.32%
CAGR of GE - 2.5%
CAGR of GOOG - 20.1%
CAGR of GOOGL - 19.72%
CAGR of GS - 10.94%
CAGR of HD - 15.16%
CAGR of HON - 8.45%
CAGR of IBM - -0.62%
CAGR of INTC - 1.06%
CAGR of INTU - 21.75%
CAGR of ISRG - 25.7%
CAGR of JNJ - 3.66%
CAGR of JPM - 13.61%
CAGR of KLAC - 27.99%
CAG

# Helper Functions

## Calculate expected returns, annual volatility and Sharpe ratio of portfolio

## Calculate CVaR of each asset

## Calculating reduction factor

## Softmax function

# Calculating Piotroski F-Score

## Helper functions for calculating Piotroski Score

In [182]:
# Getting the net income of an asset
def get_net_income(income_df, date):
  return float(income_df['netIncome'].loc[date])

# Get return on assets
"""The term return on assets (ROA) refers to a financial ratio that indicates how profitable
a company is in relation to its total assets. Corporate management, analysts, and investors
can use ROA to determine how efficiently a company uses its assets to generate a profit."""
def get_roa(balance_df, income_df, date, prev_date):
  current = float(balance_df['totalAssets'].loc[date])
  previous = float(balance_df['totalAssets'].loc[prev_date])
  av_assets = (current+previous)/2
  return get_net_income(income_df, date)/av_assets

# Get operating cash flow
def get_ocf(cash_df, date):
  return int(cash_df['operatingCashflow'].loc[date])

# Get long term debt
def get_ltdebt(balance_df, date, prev_date):
  if 'longTermDebt' in balance_df.columns:
    current = float(balance_df['longTermDebt'].loc[date])
    previous = float(balance_df['longTermDebt'].loc[prev_date])

    return previous-current

  return None

# Get current ratio
def get_current_ratio(balance_df, date, prev_date):
  current_TCA = float(balance_df['totalCurrentAssets'].loc[date])
  previous_TCA = float(balance_df['totalCurrentAssets'].loc[prev_date])
  current_TCL = float(balance_df['totalCurrentLiabilities'].loc[date])
  previous_TCL = float(balance_df['totalCurrentLiabilities'].loc[prev_date])

  ratio1 = current_TCA/current_TCL
  ratio2 = previous_TCA/previous_TCL

  return ratio1-ratio2

# Get count of new shares if issued
def get_new_shares(balance_df, date, prev_date):
  current = float(balance_df['commonStock'].loc[date])
  previous = float(balance_df['commonStock'].loc[prev_date])

  return current-previous

# Get gross margin
def get_gross_margin(income_df, date, prev_date):
  current = float(income_df['grossProfit'].loc[date])/float(income_df['totalRevenue'].loc[date])
  previous = float(income_df['grossProfit'].loc[prev_date])/float(income_df['totalRevenue'].loc[prev_date])

  return current-previous


# Get asset turnover ratio
def get_asset_turnover_ratio(income_df, balance_df, date, prev_date, prev_date2):
  current = float(balance_df['totalAssets'].loc[date])
  prev_1 = float(balance_df['totalAssets'].loc[prev_date])
  prev_2 = float(balance_df['totalAssets'].loc[prev_date2])

  av_assets1 = (current+prev_1)/2
  av_assets2 = (prev_1+prev_2)/2

  atr1 = float(income_df['totalRevenue'].loc[date])/av_assets1
  atr2 = float(income_df['totalRevenue'].loc[prev_date])/av_assets2

  return atr1-atr2

In [144]:
def get_piotroski_score(income_df, balance_df, cash_df, piotroski_asset, dates):
  for date in dates[2:]:
      print(f'Date: {date}')
      score=0
      passed = []
      index = piotroski_asset.index.get_loc(date)
      prev_date = piotroski_asset.index[index-1]
      prev_date2 = piotroski_asset.index[index-2]

      print(f"Net income - {get_net_income(income_df, date)}")

      if get_net_income(income_df, date)>0:
        passed.append("Net income")
        score+=1

      print(f'ROA - {get_roa(balance_df, income_df, date, prev_date)}')

      if get_roa(balance_df, income_df, date, prev_date)>0:
        passed.append("ROA")
        score+=1

      print(f"Operating Cash flow - {get_ocf(cash_df, date)}")

      if get_ocf(cash_df, date)>0:
        passed.append("CFO")
        score+=1

      print(f"Operating Cash flow > Net Income - {get_ocf(cash_df, date)>get_net_income(income_df, date)}")

      if get_ocf(cash_df, date)>get_net_income(income_df, date):
        passed.append("CFO > NI")
        score+=1

      print(f"Long term debt - {get_ltdebt(balance_df, date, prev_date)}")

      if get_ltdebt(balance_df, date, prev_date) is None:
        print("No data on long term debt!")

      elif get_ltdebt(balance_df, date, prev_date)>0:
        passed.append("LTD")
        score+=1

      print(f"Current ratio - {get_current_ratio(balance_df, date, prev_date)}")

      if get_current_ratio(balance_df, date, prev_date)>0:
        passed.append("CR")
        score+=1

      print(f"New shares count - {get_new_shares(balance_df, date, prev_date)}")

      if get_new_shares(balance_df, date, prev_date)>0:
        passed.append("New shares")
        score+=1

      print(f"Gross Margin - {get_gross_margin(income_df, date, prev_date)}")

      if get_gross_margin(income_df, date, prev_date)>0:
        passed.append("GM")
        score+=1

      print((f"Asset turnover ratio - {get_asset_turnover_ratio(income_df, balance_df, date, prev_date, prev_date2)}"))

      if get_asset_turnover_ratio(income_df, balance_df, date, prev_date, prev_date2)>0:
        passed.append("ATR")
        score+=1

      print(f'Passed metrics - {passed}')

      print(f'Score - {score}\n')

      piotroski_asset.loc[date] = score

  return piotroski_asset

# Calculating RoE (Return on Equity)

In [145]:
def get_roe(income_df, balance_df, roe_asset, dates):
  for date in dates[1:]:
    print(f'Date: {date}')
    index = roe_asset.index.get_loc(date)
    prev_date = roe_asset.index[index-1]
    ni = float(income_df['netIncome'].loc[date])

    eq1 = float(balance_df['totalShareholderEquity'].loc[date])
    eq2 = float(balance_df['totalStockholderEquity'].loc[prev_date])

    avg_eq = (eq1+eq2)/2
    roe = ni/avg_eq

    roe_asset.loc[date] = roe
    print(f'RoE - {roe}\n')

  return roe_asset

# Calculating RoCE (Return on Capital Employed)

In [146]:
def get_roce(income_df, balance_df, roce_asset, dates):
  # n = float(income_df.loc["Ebit"][0])
  for date in dates:
    print(f'Date: {date}')
    n = float(income_df['operatingIncome'].loc[date])
    d = float(balance_df['totalAssets'].loc[date])-float(balance_df['totalCurrentLiabilities'].loc[date])

    roce = n/d
    roce_asset.loc[date] = roce
    print(f'RoCE - {roce}\n')

  return roce_asset

# Calculating P/E Ratio

In [147]:
def get_pe(earnings_df, df, pe_asset, asset, dates):
  for date in dates:
    print(f'Date: {date}')
    if date not in df.index: continue
    pe = float(df[asset].loc[date]) / float(earnings_df['reportedEPS'].loc[date])
    pe_asset.loc[date] = pe
    print(f'P/E ratio for {asset} is {round(pe, 2)}\n')

  return pe_asset

# Calculating P/B Ratio

In [148]:
def get_pb(balance_df, df, pb_asset, asset, dates):
  for date in dates:
    print(f'Date: {date}')
    if date not in df.index: continue
    p = float(df[asset].loc[date])
    b = (float(balance_df['totalAssets'].loc[date]) - float(balance_df['intangibleAssets'].loc[date]) - float(balance_df['totalLiabilities'].loc[date])) / (float(balance_df['commonStockSharesOutstanding'].loc[date]))
    pb = p/b
    pb_asset.loc[date] = pb
    print(f'P/B ratio for {asset} is {round(pb, 2)}\n')

  return pb_asset

# Calculating PEG ratio [Not important]

# Calculating Debt-to-Asset Ratio

In [149]:
def get_debt_to_assets_ratio(balance_df, da_asset, dates):
  for date in dates:
    print(f'Date: {date}')
    da = (float(balance_df['shortTermDebt'].loc[date]) + float(balance_df['longTermDebt'].loc[date])) / float(balance_df['totalAssets'].loc[date])
    da_asset.loc[date] = da
    print(f'Debt to asset ratio - {da}\n')

  return da_asset

# Calculating CAGR (Compound Annual Growth Rate)

In [150]:
def get_cagr(ticker):
  return round(ffn.core.calc_cagr(df[ticker]) * 100, 2)

# Checking if current assets are 1.5 times the current liabilities

In [151]:
def assets_vs_liabilities(balance_df, check_asset, dates):
  for date in dates:
    print(f'Date: {date}')
    if balance_df['totalCurrentAssets'].loc[date] > (balance_df['totalCurrentLiabilities'].loc[date] * 1.5):
      check_asset.loc[date] = "Yes"
    else:
      check_asset.loc[date] = "No"

  return check_asset

# Fundamentals analyser

In [165]:
def calc_piotroski_score(tickers, dates):
  piotroski_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  piotroski_df['fiscalDateEnding'] = dates
  piotroski_df = piotroski_df.set_index('fiscalDateEnding')

  for asset in tickers:
    print(f'Asset: {asset}')
    income_df = extra_info_tickers[asset]['financials']
    balance_df = extra_info_tickers[asset]['balance_sheet']
    cash_df = extra_info_tickers[asset]['cash_flow']

    piotroski_df[asset] = get_piotroski_score(income_df, balance_df, cash_df, piotroski_df[asset], dates)

  return piotroski_df

In [153]:
def calc_cagr(tickers):
  cagr_df = pd.DataFrame(columns = tickers)
  for ticker in tickers:
    cagr = get_cagr(ticker)
    print(f'CAGR for {ticker} is {cagr}%')
    cagr_df[ticker] = cagr

  return cagr_df

In [154]:
def calc_roe(tickers, dates):
  roe_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  roe_df['fiscalDateEnding'] = dates
  roe_df = roe_df.set_index('fiscalDateEnding')

  for asset in tickers:
    income_df = extra_info_tickers[asset]['financials']
    balance_df = extra_info_tickers[asset]['balance_sheet']
    roe_df[asset] = get_roe(income_df, balance_df, roe_df[asset], dates)

  return roe_df

In [155]:
def calc_roce(tickers, dates):
  roce_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  roce_df['fiscalDateEnding'] = dates
  roce_df = roce_df.set_index('fiscalDateEnding')

  for asset in tickers:
    income_df = extra_info_tickers[asset]['financials']
    balance_df = extra_info_tickers[asset]['balance_sheet']
    roce_df[asset] = get_roce(income_df, balance_df, roce_df[asset], dates)

  return roce_df

In [156]:
def calc_pe(tickers, df, dates):
  pe_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  pe_df['fiscalDateEnding'] = dates
  pe_df = pe_df.set_index('fiscalDateEnding')

  for asset in tickers:
    earnings_df = extra_info_tickers[asset]['earnings']
    pe_df[asset] = get_pe(earnings_df, df, pe_df[asset], asset, dates)

  return pe_df

In [157]:
def calc_pb(tickers, df, dates):
  pb_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  pb_df['fiscalDateEnding'] = dates
  pb_df = pb_df.set_index('fiscalDateEnding')

  for asset in tickers:
    balance_df = extra_info_tickers[asset]['balance_sheet']
    pb_df[asset] = get_pb(balance_df, df, pb_df[asset], asset, dates)

  return pb_df

In [158]:
def calc_debt_to_assets_ratio(tickers, dates):
  da_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  da_df['fiscalDateEnding'] = dates
  da_df = da_df.set_index('fiscalDateEnding')

  for asset in tickers:
    balance_df = extra_info_tickers[asset]['balance_sheet']
    da_df[asset] = get_debt_to_assets_ratio(balance_df, da_df[asset], dates)

  return da_df

In [159]:
def check_if_assets_are_greater_than_liabilities(tickers, dates):
  check_df = pd.DataFrame(columns = ['fiscalDateEnding'] + tickers)
  check_df['fiscalDateEnding'] = dates
  check_df = check_df.set_index('fiscalDateEnding')

  for asset in tickers:
    balance_df = extra_info_tickers[asset]['balance_sheet']
    check_df[asset] = assets_vs_liabilities(balance_df, check_df[asset], dates)

  return check_df

In [160]:
fundamentals_analyser_dict = dict()

In [161]:
def fundamentals_analyser(tickers, df):
    l1 = extra_info_tickers['AAPL']['balance_sheet'].index
    l2 = extra_info_tickers['AAPL']['cash_flow'].index
    l3 = extra_info_tickers['AAPL']['earnings'].index
    l4 = extra_info_tickers['AAPL']['financials'].index

    dates = list(set(l1) & set(l2) & set(l3) & set(l4))
    dates.sort()
    pd.DatetimeIndex(dates)

    print("PIOTROSKI SCORE:\n")
    fundamentals_analyser_dict['piotroski_score'] = calc_piotroski_score(tickers, dates)
    print("--------------------------")
    print("CAGR:\n")
    fundamentals_analyser_dict['cagr'] = calc_cagr(tickers)
    print("--------------------------")
    print("RoE:\n")
    fundamentals_analyser_dict['roe'] = calc_roe(tickers, dates)
    print("--------------------------")
    print("RoCE:\n")
    fundamentals_analyser_dict['roce'] = calc_roce(tickers, dates)
    print("--------------------------")
    print("P/E ratio:\n")
    fundamentals_analyser_dict['pe_ratio'] = calc_pe(tickers, df, dates)
    # PEG RATIO NOT AVAILABLE FOR MOST ASSETS
    # print("--------------------------")
    # print("PEG ratio")
    # calc_peg(t, extra_info_tickers.tickers[t].info)
    print("--------------------------")
    print("P/B ratio")
    fundamentals_analyser_dict['pb_ratio'] = calc_pb(tickers, df, dates)
    print("--------------------------")
    print("Debt to asset ratio")
    fundamentals_analyser_dict['da_ratio'] = calc_debt_to_assets_ratio(tickers, dates)
    print("--------------------------")
    fundamentals_analyser_dict['check'] = check_if_assets_are_greater_than_liabilities(tickers, dates)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~")

In [183]:
fundamentals_analyser(assets, df)

PIOTROSKI SCORE:

Asset: AAPL
Date: 2010-03-31 00:00:00
Net income - 3074000000.0
ROA - 0.05539587143977005
Operating Cash flow - 2330000000
Operating Cash flow > Net Income - False
Long term debt - nan
Current ratio - 0.09919608693164772
New shares count - 591000000.0
Gross Margin - 0.00791094895115213
Asset turnover ratio - -0.06598456303119066
Passed metrics - ['Net income', 'ROA', 'CFO', 'CR', 'New shares', 'GM']
Score - 6

Date: 2010-06-30 00:00:00
Net income - 3253000000.0
ROA - 0.053423330213003564
Operating Cash flow - 4801000000
Operating Cash flow > Net Income - True
Long term debt - nan
Current ratio - -0.336174111776363
New shares count - 580000000.0
Gross Margin - -0.025869507672896752
Asset turnover ratio - 0.014575294056225485
Passed metrics - ['Net income', 'ROA', 'CFO', 'CFO > NI', 'New shares', 'ATR']
Score - 6

Date: 2010-09-30 00:00:00
Net income - 4308000000.0
ROA - 0.061583326185779226
Operating Cash flow - 5683000000
Operating Cash flow > Net Income - True
Long t

TypeError: cannot convert the series to <class 'int'>

In [181]:
extra_info_tickers['AAPL']['cash_flow']['operatingCashflow']

fiscalDateEnding
2024-03-31    22690000000
2023-12-31    39895000000
2023-09-30    21598000000
2023-06-30    26380000000
2023-03-31    28560000000
2023-03-31    28560000000
2022-12-31    34005000000
2022-09-30    24127000000
2022-06-30    22892000000
2022-03-31    28166000000
2021-12-31    46966000000
2021-09-30    20200000000
2021-06-30    21094000000
2021-03-31    23981000000
2020-12-31    38763000000
2020-09-30    20576000000
2020-06-30    16271000000
2020-03-31    13311000000
2019-12-31    30516000000
2019-09-30    19910000000
2019-06-30    11636000000
2019-03-31    11155000000
2018-12-31    26690000000
2018-09-30    19523000000
2018-06-30    14488000000
2018-03-31    15130000000
2017-12-31    28293000000
2017-09-30    15749000000
2017-06-30     8672000000
2017-03-31    12570000000
2016-12-31    27234000000
2016-09-30    16533000000
2016-06-30    10634000000
2016-03-31    -9490000000
2015-12-31    28548000000
2015-09-30   -43358000000
2015-06-30    10677000000
2015-03-31    1356900